In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
import pandas_profiling as pdp

In [3]:
train_data = pd.read_csv("C:/Users/Dell/Downloads/Compressed/Hacakthon/Hacakthon/train.csv")
test_data = pd.read_csv("C:/Users/Dell/Downloads/Compressed/Hacakthon/Hacakthon/test.csv")

In [4]:
train_data.columns

Index(['loan_id', 'source', 'financial_institution', 'interest_rate',
       'unpaid_principal_bal', 'loan_term', 'origination_date',
       'first_payment_date', 'loan_to_value', 'number_of_borrowers',
       'debt_to_income_ratio', 'borrower_credit_score', 'loan_purpose',
       'insurance_percent', 'co-borrower_credit_score', 'insurance_type', 'm1',
       'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12',
       'm13'],
      dtype='object')

 train_data.profile_report()

In [5]:
train_data['origination_date']=pd.to_datetime(train_data['origination_date'])

In [6]:
Labels = train_data['m13']
print(Labels)
drop_cols=['m13','loan_id','number_of_borrowers']

0         1
1         1
2         1
3         1
4         1
5         1
6         1
7         1
8         1
9         1
10        1
11        1
12        1
13        1
14        1
15        1
16        1
17        1
18        1
19        1
20        1
21        1
22        1
23        1
24        1
25        1
26        1
27        1
28        1
29        1
         ..
116028    0
116029    0
116030    0
116031    0
116032    0
116033    0
116034    0
116035    0
116036    0
116037    0
116038    0
116039    0
116040    0
116041    0
116042    0
116043    0
116044    0
116045    0
116046    0
116047    0
116048    0
116049    0
116050    0
116051    0
116052    0
116053    0
116054    0
116055    0
116056    0
116057    0
Name: m13, Length: 116058, dtype: int64


In [7]:
Labels1 = list(Labels)
np.bincount(Labels1)

array([115422,    636], dtype=int64)

In [8]:
train_data['first_payment_date']=pd.to_datetime(train_data['first_payment_date'])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=10)
fi_tfidf = vectorizer.fit_transform(train_data['financial_institution'])
train_data['source'] = train_data['source'].replace(to_replace=['X','Y','Z'],value=[0,1,2])
print(train_data['source'])
#source=  vectorizer.fit_transform(train_data['source'])
#oan_purposse= vectorizer.fit_transform(train_data['loan_purpose'])

print("Shape of matrix after one hot encodig ",fi_tfidf.shape)
#rint("Shape of matrix after one hot encodig ",loan_purposse)
from sklearn.preprocessing import StandardScaler

# price_standardized = standardScalar.fit(project_data['price'].values)
# this will rise the error
# ValueError: Expected 2D array, got 1D array instead: array=[725.05 213.03 329.   ... 399.   287.73   5.5 ].
# Reshape your data either using array.reshape(-1, 1)


0         2
1         1
2         2
3         0
4         0
5         1
6         0
7         1
8         0
9         0
10        0
11        0
12        0
13        0
14        1
15        0
16        1
17        1
18        1
19        1
20        0
21        1
22        0
23        0
24        0
25        0
26        1
27        0
28        0
29        1
         ..
116028    1
116029    0
116030    2
116031    1
116032    1
116033    0
116034    0
116035    1
116036    1
116037    0
116038    0
116039    0
116040    2
116041    1
116042    0
116043    0
116044    0
116045    0
116046    0
116047    0
116048    0
116049    1
116050    0
116051    0
116052    0
116053    1
116054    2
116055    0
116056    0
116057    2
Name: source, Length: 116058, dtype: int64
Shape of matrix after one hot encodig  (116058, 43)


In [ ]:
import numpy
dropcolus=['loan_purpose','financial_institution','origination_date','first_payment_date','loan_id','m13']
x= train_data.drop(dropcolus,axis=1)


In [ ]:
from sklearn.preprocessing import StandardScaler

# price_standardized = standardScalar.fit(project_data['price'].values)
# this will rise the error
# ValueError: Expected 2D array, got 1D array instead: array=[725.05 213.03 329.   ... 399.   287.73   5.5 ].
# Reshape your data either using array.reshape(-1, 1)

price_scalar = StandardScaler()
price_scalar.fit(x.values.reshape(-1,1)) # finding the mean and standard deviation of this data
print(f"Mean : {price_scalar.mean_[0]}, Standard deviation : {np.sqrt(price_scalar.var_[0])}")

# Now standardize the data with above maen and variance
price_standardized = price_scalar.transform(x.values.reshape(116058,23 ))

Mean : 9124.134644762702, Standard deviation : 48721.490487718


In [ ]:
from scipy.sparse import hstack
# with the same hstack function we are concatinating a sparse matrix and a dense matirx :)
X = hstack((fi_tfidf,price_standardized))
X.shape

(116058, 66)

In [ ]:
# from sklearn import svm
# clf = svm.SVC(kernel='linear')
# clf.fit(X,Labels)
# print(clf.predict(X))
# y_pred=clf.predict(X)
# for i in y_pred:
#     if (i==1):
#         print("hurray")


In [ ]:
from sklearn.cluster import KMeans
import numpy as np
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)

In [ ]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(ratio='auto')
X,Labels = smt.fit_sample(X,Labels)

Using TensorFlow backend.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
clf =RandomForestClassifier()
params = {'class_weight':[{0:neg_weight, 1:1} for neg_weight in np.arange(1.0, 5.0, 0.5)]} 
gs = GridSearchCV(estimator=clf, param_grid=params, cv=5)
gs.fit(X, Labels)


In [ ]:
model = XGBClassifier(learning_rate = 0.055,max_depth = 4)
#train_x, test_x, train_Y, test_y = train_test_split(X_train,Labels,train_size=0.8)
# rfe = RFE(model, 20)
# rfe = rfe.fit(train_x,Labels)
model.fit(X,Labels)
# print(rfe.support_)
# print(rfe.ranking_)
# xgb_predict = model.predict(X_test)

In [ ]:
y =gs.predict(X)
sklearn.metrics.f1_score(Labels, y)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=10)
fi_tfidf = vectorizer.fit_transform(test_data['financial_institution'])
test_data['source'] = test_data['source'].replace(to_replace=['X','Y','Z'],value=[0,1,2])
print(test_data['source'])
#source=  vectorizer.fit_transform(train_data['source'])
#oan_purposse= vectorizer.fit_transform(train_data['loan_purpose'])

print("Shape of matrix after one hot encodig ",fi_tfidf.shape)
#rint("Shape of matrix after one hot encodig ",loan_purposse)
from sklearn.preprocessing import StandardScaler


In [ ]:
import numpy
dropcolus=['loan_purpose','financial_institution','origination_date','first_payment_date','loan_id']
z= test_data.drop(dropcolus,axis=1)


In [ ]:
from sklearn.preprocessing import StandardScaler

# price_standardized = standardScalar.fit(project_data['price'].values)
# this will rise the error
# ValueError: Expected 2D array, got 1D array instead: array=[725.05 213.03 329.   ... 399.   287.73   5.5 ].
# Reshape your data either using array.reshape(-1, 1)

price_scalar = StandardScaler()
price_scalar.fit(x.values.reshape(-1,1)) # finding the mean and standard deviation of this data
print(f"Mean : {price_scalar.mean_[0]}, Standard deviation : {np.sqrt(price_scalar.var_[0])}")

# Now standardize the data with above maen and variance.

price_standardize = price_scalar.transform(z.values.reshape(35866,23 ))

In [ ]:
from scipy.sparse import hstack
# with the same hstack function we are concatinating a sparse matrix and a dense matirx :)
Z = hstack((fi_tfidf,price_standardize))
Z.shape

In [ ]:
pred =gs.predict(Z)
# for i in pred:
#     if(i==1):
#         print("Hurray")

In [ ]:
cols=['loan_id','m13']
output=pd.DataFrame(columns=cols)
print(output.columns)
output['loan_id']=test_data['loan_id']
output['m13']=pred
output.to_csv("pred1.csv", index=False, header=True)